SCENARIO
Your client is a multinational real estate developer that builds residential and commercial properties 
around the world. They have a large portfolio of projects that are in development simultaneously. They 
currently have a 25% failure rate for their projects that is significantly higher than the industry 
benchmark of less than 10%. They would like to understand what the key leading indicators for project 
failure are when they are planning their projects. This will allow them to only invest capital into the best 
quality projects. They also want to know ongoing which projects are likely to fail so they can cut their 
losses and cease the projects. Secondly, the real estate developer would like to search local building 
policies to quickly find the relevant answers as they plan their developments. This could include zoning 
policies, environmental policies and development planning policies.

GOAL
Predict failure! Using NYC construction data.
Which ongoing projects are likely to fail?
Which projects are worth investing in? (less likely to fail)

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://datasocibmproject.s3.ap-southeast-2.amazonaws.com/structured_data/capital_project_schedules_and_budgets_1.csv")

df.head

# in case the data frame is accidentally deleted or irreversibly altered
df_copy = df.copy()

TODO:
reassigning names and datatypes; clean missing, incorrect, duplicate values, outliers

REASSIGNING NAMES AND DATATYPES

In [20]:
list(df.columns)

['Unnamed: 0',
 'project_geographic_district',
 'project_building_identifier',
 'project_school_name',
 'project_type',
 'project_description',
 'project_phase_name',
 'project_status_name',
 'project_phase_actual_start_date',
 'project_phase_planned_end_date',
 'project_phase_actual_end_date',
 'project_budget_amount',
 'final_estimate_of_actual_costs_through_end_of_phase_amount',
 'total_phase_actual_spending_amount',
 'dsf_number_s',
 'failure']

HANDLING MISSING AND INCORRECT VALUES, DUPLICATES


In [21]:
# no duplicates found
df[df.duplicated()].shape

(0, 16)

In [25]:
# project_phase_actual_end_date

# the project_phase_actual_end_date column has many missing values, which actually corresponds to the fact that the project is in progress and is incomplete
# the project_status_name column actually captures this information so to make the dates easy to analyse, every entry that is not a date will be assigned NaN

# use pd.to_datetime to convert valid dates and coerce errors to NaT
df['project_phase_actual_end_date'] = pd.to_datetime(df['project_phase_actual_end_date'], errors='coerce')

# remove the time component 
df['project_phase_actual_end_date'] = df['project_phase_actual_end_date'].dt.strftime('%Y-%m-%d')

In [23]:
# project_phase_name and final_estimate_of_actual_costs_through_end_of_phase_amount
# these columns only have a few missing values so these rows were removed
df.dropna(subset=['final_estimate_of_actual_costs_through_end_of_phase_amount', 'project_phase_name'])


,Unnamed: 0,project_geographic_district,project_building_identifier,project_school_name,project_type,project_description,project_phase_name,project_status_name,project_phase_actual_start_date,project_phase_planned_end_date,project_phase_actual_end_date,project_budget_amount,final_estimate_of_actual_costs_through_end_of_phase_amount,total_phase_actual_spending_amount,dsf_number_s,failure
0,0,10,X254,I.S. 254 - BRONX,SCA CIP,FACADE/ROOFS,Construction,In-Progress,07/07/2016,03/28/2018,NaN,4050000,4229193.0,3791127.0,"DSF: 0000821031, 0000822210",False
1,1,10,X254,I.S. 254 - BRONX,SCA CIP,FACADE/ROOFS,"CM, F&E",In-Progress,07/07/2016,03/28/2018,NaN,252000,489326.0,485620.0,"DSF: 0000821031, 0000822210",False
2,2,10,X254,I.S. 254 - BRONX,SCA CIP RESOA,FY16 RESO A IP SURVEILLANCE CAMERA INSTALLATION,Scope,Complete,07/27/2017,11/27/2017,2017-08-03,0,1792.0,1792.0,DSF: 0000823560,False
3,3,10,X254,I.S. 254 - BRONX,SCA CIP RESOA,FY16 RESO A IP SURVEILLANCE CAMERA INSTALLATION,Design,In-Progress,08/04/2017,02/05/2018,NaN,19110,17318.0,7169.0,DSF: 0000823560,True
4,4,10,X254,I.S. 254 - BRONX,SCA CIP RESOA,FY16 RESO A IP SURVEILLANCE CAMERA INSTALLATION,Construction,PNS,PNS,PNS,NaN,280280,280280.0,951.0,DSF: 0000823560,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,8180,32,K842,P.S. 45 - BROOKLYN,DOE - Skilled Trades,REPLACE (8) UNIVENTS,Construction,In-Progress,02/09/2017,DOES,NaN,DOES,228528.0,122258.0,DSF: 0000851654,False
8181,8181,32,K865,BUSHWICK LEADERS - K,SCA CIP,CERTIFICATE OF OCCUPANCY,Scope,Complete,06/18/2013,12/02/2013,2013-11-18,93274,124725.0,118749.0,DSF: 0000776207,False
8182,8182,32,K865,BUSHWICK LEADERS - K,SCA CIP,CERTIFICATE OF OCCUPANCY,Design,In-Progress,11/19/2013,05/22/2014,NaN,373096,495448.0,338602.0,DSF: 0000776207,True
8183,8183,32,K865,BUSHWICK LEADERS - K,SCA CIP,CERTIFICATE OF OCCUPANCY,Construction,PNS,PNS,PNS,NaN,6995548,6995548.0,107204.0,DSF: 0000776207,False
